# Finite volume solver for the acoustic wave equations

## A linear system of conservation laws: The acoustic wave equation
We consider the acoustic problem
\begin{align} 
      \partial_{t} p  +  c \operatorname{div}(\mathbf{q}) &= 0 \quad \text{ in } \Omega \times I = (0,1)^2 \times (0,T), \\
      \partial_{t} \mathbf{q}  +  c \nabla p &= 0 \quad \text{ in } \Omega \times I = (0,1)^2 \times (0,T), \\
%      p (0,\cdot) &= p(1,\cdot), \quad
%      p (\cdot,0) = p(\cdot,1), \label{eq:per2b}\\
%      \mathbf{q} (0,\cdot) &= \mathbf{q}(1,\cdot), \quad
%      \mathbf{q} (\cdot,0) = \mathbf{q}(\cdot,1), \label{eq:per2bb}\\
      p &= p_0 \quad \text{ on } \Omega \times \{0\},\\
      q &= 0 \quad \text{ on } \Omega \times \{0\}.
\end{align}
Here $p$ is the acoustic pressure (the local deviation from the ambient pressure) and $\mathbf{q}$ is the local velocity. 

Boundary conditions are specified below.

The problem can be written as a system of conservation laws

$$
  \partial_t \mathbf{U} + \operatorname{div}(\mathbf{F}(\mathbf{U})) = 0 \qquad in \qquad \Omega \times[0,T]
$$

where 
* $\mathbf{U} = (p,\mathbf{q}) = (p,q_1,q_2)$ is the state variable 
* and $\mathbf{F}(\mathbf{U}) = (\mathbf{f}_1(\mathbf{U}),\mathbf{f}_2(\mathbf{U}))$ where $\mathbf{f}_i(\mathbf{U})$ are the fluxes in coordinate directions $i$ (see excercise).

## Prerequisites (libraries and helper functions)
We start loading several libraries:

In [ ]:
from math import pi
from ngsolve import *
from netgen.geom2d import SplineGeometry
ngsglobals.msg_level = 1
from ngsolve.meshes import *
from draw import *
#from netgen import gui
#from ngsolve import *

We may need function for absolute value and maxima computations on `CoefficientFunctions` which we compose with the `IfPos` function:

In [ ]:
def Abs(u):
    return IfPos(u,u,-u)

def Max(u,v):
    return IfPos(u-v,u,v)

## The mesh
For the mesh we take a N(xN) mesh. The time step is manually adjusted to provide stability in the sense of a CFL condition:

In [ ]:
N=40
mesh = MakeStructured2DMesh(nx=N,ny=N,quads=True,periodic_x=True,periodic_y=True)
dt=0.5/N

In [ ]:
dim = mesh.dim+1

## A simple finite volume solver

We can now define a very simple finite volume solver based on:
 * a flux function $F$,
 * a numerical flux function $\hat{f}_n$,
 * initial values $u_0$,
 * a mesh and
 * a time step for an explicit Euler discretization.
 
Let $u_T$ be the constant corresponding to one element. Then the scheme reads as

$$
u_T^{n+1} = u_{T}^n - \frac{\Delta t}{|T|} \sum_{K \in \partial T} 
\hat{f}_{K,n}(u_T,u_{T'},n_K) |K|
$$
    

This, together with some plotting at $t = 0,0.1,..,0.5$ is done in the following function:

In [ ]:
W = L2(mesh,order=0,dim=dim)
def Solve(F, fhatn, u0, mesh, dt):
    gfu=GridFunction(W)
    U,V = W.TnT()
    a = BilinearForm (W, nonassemble=True)
    a += InnerProduct(fhatn(F,U,U.Other(),specialcf.normal(mesh.dim)),V) * dx(element_boundary=True)
    t=0

    gfu.Set(u0)

    Ts = [0.5]
    i = 0
    for T in Ts:
        while t < T-dt/2:
            gfu.vec.data -= dt * W.InvM() @ a.mat * gfu.vec
            t += dt
            i += 1
            Redraw()
        Draw(gfu[0],mesh,"u")
    print(i,"steps")
    return gfu

## The flux function and boundary condition of the problem

In [ ]:
c = 1 # speed of propagation
def F(U):
    p, q1, q2 = U   # unpack the state vector
    return None # TODO

As initial values we load data from the provided file:

In [ ]:
import pickle
U0 = pickle.load (open("nicholas.pickle", "rb"))  
Draw(U0[0],mesh,"u",min=-1,max=1)
Draw(U0[1],mesh,"u",min=-1,max=1)
Draw(U0[2],mesh,"u",min=-1,max=1)    

## numerical examples

### A discretization with a Lax-Friedrichs flux

In [ ]:
def fhatn_LF(F,u1,u2,n):
    return None #TODO

In [ ]:
gfu = Solve(F,fhatn_LF, U0, mesh, dt)